In [1]:
print("ho")

ho

In [2]:
dbutils.fs.ls("dbfs:/FileStore/tables/data.csv")

Out[ 3 ]: [FileInfo(path=u'dbfs:/FileStore/tables/data.csv', name=u'data.csv', size=346520L)]

In [3]:
otrodf=spark.read.option("header","true").option("multiline","true").csv('dbfs:/FileStore/tables/data.csv')

In [4]:
otrodf.createOrReplaceTempView("tabledata")

In [5]:
%sql select `Mesh repair` mesh,`Laparoscopic approach` lapar,
`Inguinal hernia` hernia,
text_final
from tabledata

mesh,lapar,hernia,text_final
Yes,No,No,"Day of Surgery Arrivals Anaesthetic Types Administered - General ASA Grade : IIINotes [NONE] Procedure - (PROC_CODE) PROC_DESC Info OPEN MESH REPAIR PARA-UMBILICAL HERNIA - POAC 14.12.16 - SLEEP APNOEA CPAP - G&S ON ADMISSION - DAY CASE - GA Operation notes: Teds Flowtron IV antibiotics Curved supraumbilical incision. Hernia sac dissected from the umbilicus and surrounding tissue Sac opened empty, Neck diameter <2cm Excess extra peritoneal fat excised Haemostasis secured Small Ventralex ST Hernia Patch placed flat intraperitoneally Straps fixed with 0 PDS and excess strap excised Wound closed with 0 PDS Skin with sc 3/0 Monocryl Pressure bandage applied 40 ml of 0.5% Chirocaine infiltration"
Yes,Yes,Yes,"Day of Surgery Arrivals Types Administered - General ASA Grade : IProcedure - (PROC_CODE) PROC_DESC [RIGHT] Info LAPAROSCOPIC RIGHT INGUINAL HERNIA POAC 21.12.16 - DAY CASE - GA - SEVERE NEEDLE PHOBIC - PRONE TO FAINTING Operation note: TEDS Flowtron IV antibiotics Subumbilical incision on the rigt Sheath cut open and rectus retrated and baloon port inserted in rectorecus space 2x5 mm port (one baloon, one green in the midline) Pneumo pre-peritoneum created Blunt dissiction of pre-peritoneal space in the midline Smal directl hernial sac found and dissected Peritoneal edge dissected off the cord structure down and also laterally Small tear in the peritoneum- varase neddle used to release pneumoperitonium Haemostasiss secured 3-d large mesh placed and fixed with sorbafix Pneumo pre-peritoneum released Port withdrawn Wound closed with 1/o pds Skin with sc 3/0 monocryl 20 ml of 0.5% chirocaine infiltration Potop: Follow daycase protocol Home today when comfortable FU 3 months Trotter, Linda - Scrub 1 Johnson, Aimmi - Circulating 1 Collins, Helen - Circulating 2 Evans, Wendy - Circulating 3 Birch, Charlotte - Circulating 1 Hughes, Ffion - Theatre Assistant Jones, Angela - ODP Anaesthetics Hadi, Mr - Consultant Surgeon Patient Sent For: 04/01/2017 08:50:00 Patient Arrived: 04/01/2017 09:00:00 In Anaesthetic Room: 04/01/2017 09:10:00 Anaesthesia Start: 04/01/2017 09:11:00 In Theatre (Time on Table): 04/01/2017 09:25:00 Surgical Start: 04/01/2017 09:35:48 [NONE] [NONE] Staff [NONE"
Yes,No,No,"Theatre Visit Theatre: Block A Theatre C Glan Clwyd Hospital Caseno:XXXXXXXX Name:XXXXXXXX DOB:XXXXXXXX Clinician: XXXXXXX: Ward 6 Anaesthetic Types Administered - General Anaesthetic ASA Grade Grade III Primary Procedure - (PROC_CODE) PROC_DESC Additional Info INCISIONAL HERNIA REPAIR Theatre (Time on Table): 05/01/2017 15:36:35 Patient Sent For: 05/01/2017 15:10:01 Patient Arrived: 05/01/2017 15:10:08 In Anaesthetic Room: 05/01/2017 15:19:14 Anaesthesia Start: 05/01/2017 15:19:28 Surgical Start: 05/01/2017 15:45:14 INCISIONAL HERNIA REPAIR LA: 20ml 0.25% chirocaine I: Midline F: Para-umbilical incisional hernia containing adherent small bowel. P: Hernia sac dissected & opened. Small bowel dissected out of hernia and off surounding anteriro abdominal wall for distance of 3-4cm in all directions. Unable to enter pre-peritoneal plane due to scarring. Ventralex ST 8cm diameter patch placed intra-peritoneally. Defect closed vertically (0 prolene, sutures picking up mesh). C: 2.0 vicryl to close superficial fascia over 14ch suction drain. 2.0 ethilon interrupted mattress."
Yes,Yes,Yes,Day of Surgery ArrivalsTypes Administered - General ASA Grade Grade I Procedure - (PROC_CODE) PROC_DESC [RIGHT] Info LAPAROSCOPIC/OPEN REPAIR RIGHT FEMORAL HERNIA. DAY CASE Procedures [NONE] Patient Sent For: 06/01/2017 10:44:08 Patient Arrived: 06/01/2017 10:50:00 In Anaesthetic Room: 06/01/2017 11:11:48 Anaesthesia Start: 06/01/2017 11:14:29 In Theatre (Time on Table): 06/01/2017 11:44:48 Surgical Start: 06/01/2017 11:55:00 Right Femoral hernia on ultrasound scan. Hasson port infraumbilical right of midline. Two x 5 mm ports. Preperitoneal dissection. Small direct hernia noted. No femoral hernia Hernia reduced. Bard 3D max mesh 

In [6]:
mydf= spark.sql("""
select `Mesh repair` mesh,`Laparoscopic approach` lapar,
`Inguinal hernia` hernia,concat(`Mesh repair`,`Laparoscopic approach`,`Inguinal hernia`) concatenated,
text_final
from tabledata
""")

In [7]:
import nltk
import pandas as pd
import re
from pyspark.ml.feature import StopWordsRemover
import string
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
remover = StopWordsRemover()
stopwords = remover.getStopWords() 
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!

In [8]:
data = mydf.toPandas()

In [9]:
def clean_text(text):
    #text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', str(text).lower())
    text = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords]
    text = [word for word in text if  not word.isdigit()]
    text = [word for word in text if   len(word)>0]
    #text = [word for word in text if  word not in string.punctuation]
    #text = [wn.lemmatize(word) for word in text]
    #text = [isnumber(word) for word in text]
    return text

In [10]:
def clean_text_not_token(text):
    #text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', str(text).lower())
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    text = [word for word in text if  not word.isdigit()]
    text = [word for word in text if  word not in string.punctuation]
    text = [wn.lemmatize(word) for word in text]
    text1 = " ".join(text)
    return text1

In [11]:
data['textfinaltoken'] = data['text_final'].apply(lambda x: clean_text(x))
data['cleantext'] = data['text_final'].apply(lambda x: clean_text(x))

In [12]:
data.to_csv('/dbfs/temp/words.csv',index=False)

In [13]:
otrodf=spark.read.option("header","true").option("multiline","true").csv('dbfs:/temp/words.csv')

In [14]:
#from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
#from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
#regexTokenizer = RegexTokenizer(inputCol="Descript", outputCol="words", pattern="\\W")
# stop words
#add_stopwords = ["http","https","amp","rt","t","c","the"] 
#stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# bag of words count
#countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [15]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer

regexTokenizer = RegexTokenizer(inputCol="cleantext", outputCol="words", pattern="\\W")
label_stringIdx = StringIndexer(inputCol = "concatenated", outputCol = "label")
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)
pipeline = Pipeline(stages=[regexTokenizer,cv, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(otrodf)
dataset = pipelineFit.transform(otrodf)



In [17]:
dataset.show(50)

+----+-----+------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
mesh|lapar|hernia|concatenated| text_final| textfinaltoken| cleantext| words| features|label|
+----+-----+------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[8.0,3...| 2.0|
 Yes| Yes| Yes| YesYesYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[8.0,1...| 0.0|
 Yes| No| No| YesNoNo|Theatre Visit The...|['theatre', 'visi...|['theatre', 'visi...|[theatre, visit, ...| (3,[0,1],[3.0,5.0])| 2.0|
 Yes| Yes| Yes| YesYesYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, ar...|(3,[0,1,2],[7.0,5...| 0.0|
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[5.0,2...| 2.0|
 No| No| No| NoNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[4.0,1...| 3.0|
 Yes| No| No| YesNoNo| Ward 6 Anaesthet...|['ward', 'anaesth...|['ward', 'anaesth...|[ward, anaestheti...| (3,[0,1],[4.0,4.0])| 2.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[4.0,8.0])| 1.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[2.0,7.0])| 1.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[5.0,3...| 1.0|
 Yes| Yes| No| YesYesNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[4.0,2...| 5.0|
 Yes| Yes| Yes| YesYesYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[6.0,2...| 0.0|
 Yes| Yes| Yes| YesYesYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[10.0,...| 0.0|
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[6.0,1...| 2.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,2],[4.0,1.0])| 1.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[4.0,2...| 1.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[4.0,3...| 1.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[4.0,7.0])| 1.0|
 Yes| No| No| YesNoNo|TTheatre Visit Th...|['ttheatre', 'vis...|['ttheatre', 'vis...|[ttheatre, visit,...| (3,[0,1],[4.0,6.0])| 2.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[4.0,4...| 1.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[5.0,4.0])| 1.0|
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[3.0,3.0])| 2.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...|(3,[0,1,2],[5.0,3...| 1.0|
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[3.0,4.0])| 2.0|
 Yes| No| Yes| YesNoYes| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[4.0,2.0])| 1.0|
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, u,...| (3,[0,1],[5.0,9.0])| 2.0|
 Yes| No| No| YesNoNo| Day of Surgery A...|['day', 'surgery'...|['day', 'surgery'...|[day, surgery, ar...|(3,[0,1,2],[6.0,

In [18]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 170
Test Dataset Count: 74

In [19]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
#predictions.filter(predictions['prediction'] == 0) \
#    .select("Descript","Category","probability","label","prediction") \
#    .orderBy("probability", ascending=False) \
#    .show(n = 10, truncate = 30)

In [20]:
display(predictions)

mesh,lapar,hernia,concatenated,text_final,textfinaltoken,cleantext,words,features,label,rawPrediction,probability,prediction
No,No,No,NoNoNo,"Day of Surgery Arrivals - 1st Floor Types Administered - General - Spinal ASA Grade Grade II Notes [NONE] Procedure - (PROC_CODE) PROC_DESC OPEN REPAIR OF INCISIONAL HERNIA USING SUTURES Old midline re-opened Several hernia sacs along the incision containing loops of small bowel with adhesions Adhesiolysis - no enterotomies Sacs excised Abdominal wall and sheath mobilised, re-freshed and closed with PDS 0 loops x2 & interrupted Ethibond 5 sutures (tension-free) Mesh not needed as mass closure achieved tension free Suction drains x2 outside the andominal wall Skin closed with Monocryl 3/0 sc POST-OP Free oral diet Chest physio Analgesia DVT px Keep drains in Info ABDOMINAL WALL RECONSTRUCTION WITH A LARGE PERMACOL MESH GA - INPATIENT (G&S ON ADMISSION) Procedures [NONE] Marked By [NONE] [NONE] Reason [NONE] Reason [NONE]","['day', 'surgery', u'arrival', '1st', 'floor', u'type', 'administered', 'general', 'spinal', 'asa', 'grade', 'grade', 'ii', u'note', 'none', 'procedure', 'proc_code', 'proc_desc', 'open', 'repair', 'incisional', 'hernia', 'using', u'suture', 'old', 'midline', 're', 'opened', 'several', 'hernia', u'sac', 'along', 'incision', 'containing', u'loop', 'small', 'bowel', u'adhesion', 'adhesiolysis', u'enterotomy', u'sac', 'excised', 'abdominal', 'wall', 'sheath', 'mobilised', 're', 'freshed', 'closed', u'pd', u'loop', 'x2', 'interrupted', 'ethibond', u'suture', 'tension', 'free', 'mesh', 'needed', 'mass', 'closure', 'achieved', 'tension', 'free', 'suction', u'drain', 'x2', 'outside', 'andominal', 'wall', 'skin', 'closed', 'monocryl', 'sc', 'post', 'op', 'free', 'oral', 'diet', 'chest', 'physio', 'analgesia', 'dvt', 'px', 'keep', u'drain', 'info', 'abdominal', 'wall', 'reconstruction', 'large', 'permacol', 'mesh', 'ga', 'inpatient', 'g', 'admission', u'procedure', 'none', 'marked', 'none', 'none', 'reason', 'none', 'reason', 'none']","['day', 'surgery', u'arrival', '1st', 'floor', u'type', 'administered', 'general', 'spinal', 'asa', 'grade', 'grade', 'ii', u'note', 'none', 'procedure', 'proc_code', 'proc_desc', 'open', 'repair', 'incisional', 'hernia', 'using', u'suture', 'old', 'midline', 're', 'opened', 'several', 'hernia', u'sac', 'along', 'incision', 'containing', u'loop', 'small', 'bowel', u'adhesion', 'adhesiolysis', u'enterotomy', u'sac', 'excised', 'abdominal', 'wall', 'sheath', 'mobilised', 're', 'freshed', 'closed', u'pd', u'loop', 'x2', 'interrupted', 'ethibond', u'suture', 'tension', 'free', 'mesh', 'needed', 'mass', 'closure', 'achieved', 'tension', 'free', 'suction', u'drain', 'x2', 'outside', 'andominal', 'wall', 'skin', 'closed', 'monocryl', 'sc', 'post', 'op', 'free', 'oral', 'diet', 'chest', 'physio', 'analgesia', 'dvt', 'px', 'keep', u'drain', 'info', 'abdominal', 'wall', 'reconstruction', 'large', 'permacol', 'mesh', 'ga', 'inpatient', 'g', 'admission', u'procedure', 'none', 'marked', 'none', 'none', 'reason', 'none', 'reason', 'none']","List(day, surgery, u, arrival, 1st, floor, u, type, administered, general, spinal, asa, grade, grade, ii, u, note, none, procedure, proc_code, proc_desc, open, repair, incisional, hernia, using, u, suture, old, midline, re, opened, several, hernia, u, sac, along, incision, containing, u, loop, small, bowel, u, adhesion, adhesiolysis, u, enterotomy, u, sac, excised, abdominal, wall, sheath, mobilised, re, freshed, closed, u, pd, u, loop, x2, interrupted, ethibond, u, suture, tension, free, mesh, needed, mass, closure, achieved, tension, free, suction, u, drain, x2, outside, andominal, wall, skin, closed, monocryl, sc, post, op, free, oral, diet, chest, physio, analgesia, dvt, px, keep, u, drain, info, abdominal, wall, reconstruction, large, permacol, mesh, ga, inpatient, g, admission, u, procedure, none, marked, none, none, reason, none, reason, none)","List(0, 3, List(0, 1, 2), List(15.0, 2.0, 6.0))",3.0,"List(1, 6, List(), 

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 81 ]: 0.29575864985701056

In [22]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
             .addGrid(lr.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 46 ]: 0.5041412954456432

In [23]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 82 ]: 0.2417575961054222